In [15]:
from itertools import permutations
import random
import math
from numpy.random import randint
from numpy.random import rand

# ALGORITMO GENETICO

### GENERAZIONE POPOLAZIONE INIZIALE

In [ ]:
A = [('a1',9), ('a2',6), ('a3',3), ('a4',8), ('a5',5)]

B = [('b1',5), ('b2',7), ('b3',2), ('b4',10)]

jobs = A + B


In [ ]:
def generate_population(jobs):
    permutazioni = list(permutations(jobs))
    return permutazioni

### FITNESS FUNCTION

In [ ]:
# fitness function equivale alla nostra funzione obbiettivo (che deve essere minimizzata)
def fitness_function(soluzione, alfa=0.5):
    somma_A = 0
    somma_B = 0
    somma_tot = 0
    for job in soluzione:
        string = str(job[0])
        somma_tot += job[1]
        if string.startswith('a'):
            somma_A += somma_tot
        else:
            somma_B += somma_tot
    
    return (alfa * (somma_A + somma_B)) + ((1-alfa) * abs((somma_A - somma_B)))

### SELEZIONE

In [ ]:
# tournament selection
def selection(population, scores, k=3):
    # primo scelto randomicamente (utile estrarlo prima per fare i confronti)
    selection_ix = randint(len(population))
    # scegliamo gli altri k-1 partecipanti randomicamente
    for ix in randint(0, len(population), k-1):
        # controlliamo chi è il migliore tra i k partecipanti
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return population[selection_ix]

### CROSSOVER

In [ ]:
def crossover(p1, p2, r_cross):
    # i figli sono copie dei genitori di default
    c1, c2 = p1.copy(), p2.copy()

    if rand() < r_cross:
        # selezioniamo il punto di taglio per il crossover
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] 
        for elem in p2:
          if elem not in c1:
             c1.append(elem)

        c2 = p2[:pt] 
        for elem in p1:
          if elem not in c2:
             c2.append(elem)
    return [c1, c2]

### MUTAZIONE

In [ ]:
# mutazione: scambio di posizione di due job
def mutation(c, r_mut):
        c1 = c[1]
        c2 = c[2]
        if rand() < r_mut:
             # Selezione casuale di due indici di due job in c1
             indexes = random.sample(range(len(c1)), 2)
             i, j = indexes[0], indexes[1]
             # Scambia gli elementi di posizione
             c1[i], c1[j] = c1[j], c1[i]

             # Selezione casuale di due indici di due job in c2
             indexes = random.sample(range(len(c2)), 2)
             i, j = indexes[0], indexes[1]
             # Scambia gli elementi di posizione
             c1[i], c1[j] = c1[j], c1[i]




### ALGORITMO

In [ ]:
def genetic_algorithm(jobs, fitness_fuction, n_iter, n_pop, r_cross, r_mut):
    # popolazione iniziale
    population = generate_population(jobs)
    # teniamo traccia della migliore soluzione (all'inizio prendiamo la prima soluzione)
    best, best_eval = 0, fitness_fuction(population[0])
    # enumeriamo le iterazioni (il numero di generazioni create)
    for gen in range(n_iter):
        # valutiamo tutti i candidati della popolazione
        scores = [fitness_fuction(solution) for solution in population]
        # vediamo qual'è la migliore soluzione della generazione corrente
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  population[i], scores[i]))
        # applichiamo la selezione: selezioniamo i genitori della nuova generazione (una coppia di genitori per ogni iterazione)
        selected = [selection(population, scores) for _ in range(n_pop)]
        # creiamo la nuova generazione
        children = list()
        # per ogni coppia di genitori
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            # crossover
            # c è una coppia di figli
            for c in crossover(p1, p2, r_cross):
                # mutazione
                mutation(c, r_mut)
                # memorizziamo la nuova generazione
                children.append(c)
        # sostituiamo la nuova generazione
        population = children
    return [best, best_eval]